In [1]:
import os
import json
import copy
import numpy as np
import pandas as pd
from pathlib import Path

In [2]:
import sys
sys.dont_write_bytecode = True
sys.path.append('../')

In [3]:
from datatools.analyzer import *

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [4]:
path = "../hand_labeled/"
datalist = ['DCM', 'DIT', 'IRS']
error_types = ['Unclear intention', 'Wrong information',
 'Ignore question', 'Topic transition error', 
 'Lack of information', 'Repetition', 
 'Contradiction', 'Self-contradiction',
  'Lack of common sense', 'Semantic error',
   'Grammatical error', 'Ignore proposal', 
   'Ignore offer', 'Lack of sociality', 
   'Uninterpretable', 'Ignore greeting', 
   'No-Err']

In [5]:
convs = read_conv(path, datalist)

In [8]:
from pyknp import Juman
from sentence_transformers import SentenceTransformer
import scipy.spatial
model_path = "/home/yamada/Downloads/training_bert_japanese"
Nmodel = SentenceTransformer(model_path, show_progress_bar=False)
emb_dim = Nmodel.encode(["お辞儀をしている男性会社員"])[0].shape[0]

In [15]:
error = "Repetition"
import csv
import Levenshtein
import random
def make_X_y_csv(filename="repetition.csv"):
    X1 = []
    X2 = []
    y = []
    all_data = []
    with open(filename, "r") as f:
        data_ = csv.reader(f)
        for d in data_:
          all_data.append(d)
    
    for d in all_data:
        y.append(int(d[0]))
        X1.append(d[1])
        X2.append(d[2])

    # u1_l = [d[1]  for d in all_data]
    # u2_l = [d[2]  for d in all_data]
    X1_vec = Nmodel.encode(X1)
    X2_vec = Nmodel.encode(X2)
    X = [ x1-x2 for x1, x2 in zip(X1_vec, X2_vec) ]
    for x1, x2 in zip( random.choices(X1_vec, k=len(X1)), random.choices(X2_vec, k=len(X2)) ):
        X.append(x1-x2)
        y.append(0)
    return np.asarray(X), np.asarray(y)

In [18]:
X, y = make_X_y_csv()

[' わあ！いいですね！', ' 映画が楽しみです', ' 海は素晴らしいですね', ' 今は、してないですね。', ' 食べるのがいいですね', ' 歯ごたえが抜群にいいですね', ' スイカは大好きですね', ' メロンは好きですね', ' 泳ぐを繰り返すのです', ' 泳ぐを覚えるのです', ' 泳ぐを繰り返すようですよ', ' 泳ぐを覚えますよねー', ' 猛暑は欲しいですよねー', ' 冷夏が多いです', ' 元気ですかは元気か', ' 熱中症で死者が出てるんでしょうか', ' 実写映画を含めるのです', ' 実写映画を問わないなあ', ' ありがとうございます', ' うんどういたしまして', ' ゴーヤチャンプルは美味しいですね', ' 予防を怠ります', ' 普段から予防を行いますよねー', ' 海は楽しいですね', ' 湖が綺麗ですね', ' 果物が良いですね', ' ありがとう', ' スイカは大好きですね', ' 趣味は写真を撮るのです', ' 機嫌をなおすのです', ' 嫌はいいのが救いですけどねぇ', ' 元気ですかは元気ですね', ' ありがとうございます', ' 代替わりしていますね', ' あはは本当ですね。（←大阪の友達に怒られそう…）東京のもんじゃも似てますけどね。', ' こんにちは。', ' 好きな食べ物は？', ' そうですねえ', ' ペットは飼ってませんが動物は好きです。', ' 最近だと旅行にはまりだしたので、旅代で消えてしまいますね。', ' ヨガちょっとやってみたいです！やっぱりやり始めてからは体の調子いいですか？お風呂上がりにストレッチやろうと思うのですが、なかなか続かなくて。', ' 猫は好きですか。', ' いま住んでるところから駅がちょっと遠いので駅までは車ですよ。', " 慣れてる方ですね'", ' 当たらないだろうと思って出していません。', ' 地下鉄で。名大前ができてすごく便利になりましたね、ここの大学。']


In [24]:
y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0])

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=5)

In [21]:
lr = LogisticRegression(solver='sag', max_iter=10000)
lr.fit(X_train, y_train)

LogisticRegression(max_iter=10000, solver='sag')

In [22]:
y_pred = lr.predict(X_test)

In [30]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
print('confusion matrix = \n', confusion_matrix(y_true=y_test, y_pred=y_pred))
print('accuracy = ', accuracy_score(y_true=y_test, y_pred=y_pred))
print('precision = ', precision_score(y_true=y_test, y_pred=y_pred))
print('recall = ', recall_score(y_true=y_test, y_pred=y_pred))
print('f1 score = ', f1_score(y_true=y_test, y_pred=y_pred))

confusion matrix = 
 [[11  1]
 [ 0 16]]
accuracy =  0.9642857142857143
precision =  0.9411764705882353
recall =  1.0
f1 score =  0.9696969696969697


In [25]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier()
clf.fit(X_train, y_train)

AdaBoostClassifier()

In [29]:
y_pred = clf.predict(X_test)

In [35]:
from tqdm import tqdm
y_pred = []
for conv in tqdm(convs):
    ngram_sets = []
    history = []
    for ut in conv:
        if not ut.is_system():
            continue
        utt = ut.utt
        doc = nlp(utt)
        y_pred.append(0)
        for sent in doc.sents:
            if len(sent) <= 3:
                # print(sent)
                continue
            vec = Nmodel.encode(sent.text)[0]
            for prev in history:
                if clf.predict(vec.reshape(1, -1)) == 1:
                    y_pred[-1] = 1
            history.append(vec)
        
            

100%|██████████| 200/200 [10:40<00:00,  3.20s/it]


In [36]:
error = "Repetition"
y = []
for conv in convs:
    for ut in conv:
        if not ut.is_system():
            continue
        # 
        if ut.is_error_included(error):
            # print(ut.errors)
            y.append(1)
        else:
            y.append(0)

In [37]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn import metrics
print('confusion matrix = \n', confusion_matrix(y_true=y, y_pred=y_pred))
print('accuracy = ', accuracy_score(y_true=y, y_pred=y_pred))
print('F-measure: ', metrics.f1_score(y, y_pred))

confusion matrix = 
 [[1788  364]
 [  38   10]]
accuracy =  0.8172727272727273
F-measure:  0.04739336492890995
